# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [1]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="./7-2_content/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

/home/youngwon/anaconda3/envs/nlp_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-16 16:37:00.683197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 16:37:01.184772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


downstream_model_checkpoint_fpath: ./7-2_content/checkpoint-qa/epoch=0-val_loss=0.47.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [2]:
import torch
from transformers import BertConfig, BertForQuestionAnswering
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [4]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [5]:
inference_fn(question = "BERT는 어디서 개발했는가?", context = "반면에 BERT (Bidirectional Encoder Representations from Transformers)는 Google에서 개발한 언어 모델로, Transformer 아키텍처를 기반으로 합니다. GPT와는 달리 BERT는 왼쪽에서 오른쪽 방향뿐만 아니라 오른쪽에서 왼쪽 방향으로도 텍스트를 처리할 수 있는 양방향 모델입니다. BERT는 자연어 이해, 감정 분석 및 질문-답변과 같은 작업에 자주 사용됩니다." )

{'question': 'BERT는 어디서 개발했는가?',
 'context': '반면에 BERT (Bidirectional Encoder Representations from Transformers)는 Google에서 개발한 언어 모델로, Transformer 아키텍처를 기반으로 합니다. GPT와는 달리 BERT는 왼쪽에서 오른쪽 방향뿐만 아니라 오른쪽에서 왼쪽 방향으로도 텍스트를 처리할 수 있는 양방향 모델입니다. BERT는 자연어 이해, 감정 분석 및 질문-답변과 같은 작업에 자주 사용됩니다.',
 'answer': 'Google'}

In [6]:
inference_fn(question = "온라인 여행에는 어떤 한계가 있는가?", context = "올해는 코로나19로 인해 여행계획이 많이 무산되고 있습니다. 그러나 여행을 좋아하는 사람이라면 온라인 여행을 즐길 수 있는 방법들이 많이 생겨났습니다. 예를 들어, 가상현실 기술을 이용한 가상여행, 인터넷으로 실제 여행지의 모습을 볼 수 있는 웹캠 등이 있습니다. 이러한 기술들을 이용하여 우리는 집에서도 여행의 느낌을 즐길 수 있습니다. 또한, 온라인으로 예약을 하고 배송을 받을 수 있는 여행 상품도 많이 출시되었습니다. 이러한 상품들을 이용하여 우리는 집에서도 다양한 여행을 즐길 수 있습니다. 그러나 이러한 온라인 여행도 결국은 실제 여행만큼 완벽한 경험을 제공하지는 못한다는 한계가 있기 때문에, 우리는 이러한 온라인 여행을 보조적인 수단으로 활용하면서도 실제 여행의 가치와 중요성을 잊지 않아야 합니다.")

{'question': '온라인 여행에는 어떤 한계가 있는가?',
 'context': '올해는 코로나19로 인해 여행계획이 많이 무산되고 있습니다. 그러나 여행을 좋아하는 사람이라면 온라인 여행을 즐길 수 있는 방법들이 많이 생겨났습니다. 예를 들어, 가상현실 기술을 이용한 가상여행, 인터넷으로 실제 여행지의 모습을 볼 수 있는 웹캠 등이 있습니다. 이러한 기술들을 이용하여 우리는 집에서도 여행의 느낌을 즐길 수 있습니다. 또한, 온라인으로 예약을 하고 배송을 받을 수 있는 여행 상품도 많이 출시되었습니다. 이러한 상품들을 이용하여 우리는 집에서도 다양한 여행을 즐길 수 있습니다. 그러나 이러한 온라인 여행도 결국은 실제 여행만큼 완벽한 경험을 제공하지는 못한다는 한계가 있기 때문에, 우리는 이러한 온라인 여행을 보조적인 수단으로 활용하면서도 실제 여행의 가치와 중요성을 잊지 않아야 합니다.',
 'answer': '완벽한 경험을 제공하지는 못한다는 한계가'}